# The financial portfolio case

## Bank Python code

### Initialization 

In [14]:
import numpy as np

In [8]:
# Definimos los parámetros fundamentales de nuestro modelo
risk_aversion = 1.0   # Risk aversion en el enfoque mean-variance
rho = 0.1             # Multiplicador de Lagrange para el término de costes
A = 1.0               # Multiplicador del término de normalización en el problema QUBO
n_digits = 4          # Numero de dígitos que utilizaremos para describir el peso de cada asset
n_assets = 14         # Número de assets que consideramos
n_intervals = 2       # Número de instantes de tiempo
n_variables = n_assets * n_digits * n_intervals # Número de variables del problema
print('numero de variables del problema: ', n_variables)

numero de variables del problema:  112


So, there are 3 types of variables:

{digit_0, digit_1, digit_2, digit_3 }<br><br>
{asset_0, asset_1, ...,  asset_13} <br><br>
{time_0, time_1} 


In [138]:
J = {}
h = {}
bias = 0.0

In [15]:
# Vectores de medias de retornos
means = np.random.randn(n_intervals, n_assets)

### Returns contribution

In [132]:
# Este bucle calcula la contribución de la media de los retornos
for time in range(n_intervals):
    for asset in range(n_assets):
        for digit in range(n_digits):
            h[time*n_digits*n_assets + asset*n_digits + digit] = 2**(-digit) * -means[time, asset]


<br>
<br>
<br>

$\sum_{\text{time}}\sum_{\text{asset}}\sum_{\text{digit}}f(time, asset, digit)\text{    }\text{X}_{time, asset, digit}$

<br>
<br>

$f(time, asset, digit) = \frac{-1}{{2}^{\text{digit }}} \text{means}(time, asset)$

In [128]:

time = []

for _ in range(n_intervals):
    time.append(_)




digit = []

for _ in range(n_digits):
    digit.append(_)

    
asset = []

for _ in range(n_assets):
    asset.append(_) 



t = time
y = asset
z = digit


from collections import defaultdict

x = defaultdict(lambda: 0.)  # user variable interactions

Q = defaultdict(lambda: 0.)


for i1 in t:
    for j1 in y:
        for k1 in z:    
                index_i = i1*len(y)*len(z) + j1*len(z) + k1
                for i2 in t:
                    for j2 in y:
                        for k2 in z:
                            if i1!=i2: continue
                            if j1!=j2: continue
                            if k1!=k2: continue
                            index_j = i2*len(y)*len(z) + j2*len(z) + k2
                            x[(i1,i2,j1,j2,k1,k2)] = (-1 * 2 ** - k1) * means[i1,j1]  # k1 = k2 due the "if k1!=k2: continue"  
                            Q[(index_i, index_j)] = (-1 * 2 ** - k1) * means[i1,j1]   # and the same for i and j  
                    

print("this is the input dictionary for DWave:")
                
#print(Q)
                   
print("this is the dictionary for the user:")
print("order of the basis: (time time', asset, asset', digit, digit' )")              

#print(x)
    

this is the input dictionary for DWave:
this is the dictionary for the user:
order of the basis: (time time', asset, asset', digit, digit' )


# Normalization contribution

In [26]:
# Contribución a bias de la normalización (no es necesaria)
bias += A

### Normalization linear contribution

In [133]:
# Este bucle lleva la contribución lineal de la normalización
for time in range(n_intervals):
    for asset in range(n_assets):
        for digit in range(n_digits):
            h[time*n_digits*n_assets + asset*n_digits + digit] += A * (2**(-digit) - 1) * 2**(-digit)
print(h)           

{0: 0.48804778698972756, 1: -0.0059761065051362194, 2: -0.06548805325256811, 3: -0.048369026626284055, 4: 1.2240869866248285, 5: 0.36204349331241426, 6: 0.11852174665620713, 7: 0.043635873328103564, 8: -0.07169990537098274, 9: -0.28584995268549135, 10: -0.20542497634274567, 11: -0.11833748817137284, 12: 1.0520194034877233, 13: 0.27600970174386164, 14: 0.07550485087193082, 15: 0.02212742543596541, 16: -1.1198176539709883, 17: -0.8099088269854942, 18: -0.4674544134927471, 19: -0.24935220674637354, 20: 1.5340379295867, 21: 0.51701896479335, 22: 0.19600948239667498, 23: 0.08237974119833749, 24: 0.8174138607162544, 25: 0.15870693035812722, 26: 0.01685346517906361, 27: -0.007198267410468195, 28: 0.399403672397387, 29: -0.0502981638013065, 30: -0.08764908190065325, 31: -0.059449540950326625, 32: -1.926784483865328, 33: -1.213392241932664, 34: -0.669196120966332, 35: -0.350223060483166, 36: -2.031114083817223, 37: -1.2655570419086115, 38: -0.6952785209543058, 39: -0.3632642604771529, 40: -1.18


<br>
<br>
<br>

$\sum_{\text{time}}\sum_{\text{asset}}\sum_{\text{digit}}f(digit)\text{    }\text{X}_{time, asset, digit}$

<br>
<br>

$f(digit) = \left ( \frac{1}{{2}^{\text{digit }}}-1 \right )\frac{A}{{2}^{\text{digit }}} $

In [129]:
for i1 in t:
    for j1 in y:
        for k1 in z:    
                index_i = i1*len(y)*len(z) + j1*len(z) + k1
                for i2 in t:
                    for j2 in y:
                        for k2 in z:
                            if i1!=i2: continue
                            if j1!=j2: continue
                            if k1!=k2: continue
                            index_j = i2*len(y)*len(z) + j2*len(z) + k2
                            x[(i1,i2,j1,j2,k1,k2)] += A * (2**(-k1) - 1) * 2**(-k1)    
                            Q[(index_i, index_j)] += A * (2**(-k1) - 1) * 2**(-k1)     
                    

print("this is the input dictionary for DWave:")
                
print(Q)
                   
print("this is the dictionary for the user:")
print("order of the basis: (time time', asset, asset', digit, digit' )")              

#print(x)
    

this is the input dictionary for DWave:
defaultdict(<function <lambda> at 0x000001D8E3BA8B80>, {(0, 0): 0.48804778698972756, (1, 1): -0.0059761065051362194, (2, 2): -0.06548805325256811, (3, 3): -0.048369026626284055, (4, 4): 1.2240869866248285, (5, 5): 0.36204349331241426, (6, 6): 0.11852174665620713, (7, 7): 0.043635873328103564, (8, 8): -0.07169990537098274, (9, 9): -0.28584995268549135, (10, 10): -0.20542497634274567, (11, 11): -0.11833748817137284, (12, 12): 1.0520194034877233, (13, 13): 0.27600970174386164, (14, 14): 0.07550485087193082, (15, 15): 0.02212742543596541, (16, 16): -1.1198176539709883, (17, 17): -0.8099088269854942, (18, 18): -0.4674544134927471, (19, 19): -0.24935220674637354, (20, 20): 1.5340379295867, (21, 21): 0.51701896479335, (22, 22): 0.19600948239667498, (23, 23): 0.08237974119833749, (24, 24): 0.8174138607162544, (25, 25): 0.15870693035812722, (26, 26): 0.01685346517906361, (27, 27): -0.007198267410468195, (28, 28): 0.399403672397387, (29, 29): -0.0502981638

### Normalization quadratic contribution

In [139]:
# Este bucle lleva la contribucion cuadratica de la normalizacion
for time in range(n_intervals):
    for asset1 in range(n_assets):
        for digit1 in range(n_digits):
            for asset2 in range(n_assets):
                for digit2 in range(n_digits):
                    if asset1 >= asset2 and digit1 >= digit2: continue
                    coord1 = time*n_digits*n_assets + asset1*n_digits + digit1
                    coord2 = time*n_digits*n_assets + asset2*n_digits + digit2
                    J[(coord1,coord2,time,time,asset1,asset2,digit1,digit2)] = A * 2**(-digit1) * 2**(-digit2)
print(J)

           
           

{(0, 1, 0, 0, 0, 0, 0, 1): 0.5, (0, 2, 0, 0, 0, 0, 0, 2): 0.25, (0, 3, 0, 0, 0, 0, 0, 3): 0.125, (0, 4, 0, 0, 0, 1, 0, 0): 1.0, (0, 5, 0, 0, 0, 1, 0, 1): 0.5, (0, 6, 0, 0, 0, 1, 0, 2): 0.25, (0, 7, 0, 0, 0, 1, 0, 3): 0.125, (0, 8, 0, 0, 0, 2, 0, 0): 1.0, (0, 9, 0, 0, 0, 2, 0, 1): 0.5, (0, 10, 0, 0, 0, 2, 0, 2): 0.25, (0, 11, 0, 0, 0, 2, 0, 3): 0.125, (0, 12, 0, 0, 0, 3, 0, 0): 1.0, (0, 13, 0, 0, 0, 3, 0, 1): 0.5, (0, 14, 0, 0, 0, 3, 0, 2): 0.25, (0, 15, 0, 0, 0, 3, 0, 3): 0.125, (0, 16, 0, 0, 0, 4, 0, 0): 1.0, (0, 17, 0, 0, 0, 4, 0, 1): 0.5, (0, 18, 0, 0, 0, 4, 0, 2): 0.25, (0, 19, 0, 0, 0, 4, 0, 3): 0.125, (0, 20, 0, 0, 0, 5, 0, 0): 1.0, (0, 21, 0, 0, 0, 5, 0, 1): 0.5, (0, 22, 0, 0, 0, 5, 0, 2): 0.25, (0, 23, 0, 0, 0, 5, 0, 3): 0.125, (0, 24, 0, 0, 0, 6, 0, 0): 1.0, (0, 25, 0, 0, 0, 6, 0, 1): 0.5, (0, 26, 0, 0, 0, 6, 0, 2): 0.25, (0, 27, 0, 0, 0, 6, 0, 3): 0.125, (0, 28, 0, 0, 0, 7, 0, 0): 1.0, (0, 29, 0, 0, 0, 7, 0, 1): 0.5, (0, 30, 0, 0, 0, 7, 0, 2): 0.25, (0, 31, 0, 0, 0, 7, 0, 3):


<br>
<br>
<br>

$\sum_{\text{time}}\sum_{\text{asset2>asset1}}\sum_{\text{digit2>digit1}}f(digit1,digit2)\text{    }\text{X}_{time, asset1, digit1}\text{X}_{time, asset2, digit2}$

<br>
<br>

$f(digit1, digit2) = \frac{A}{{2}^{\text{digit1 + digit2 }}} $

In [130]:
for i1 in t:
        for j1 in y:
            for k1 in z:
                for i2 in t:
                    for j2 in y:
                        for k2 in z:
                            if i1!=i2: continue
                            if j1>=j2: continue
                            if k1>=k2: continue
                            index_i = i1*len(y)*len(z) + j1*len(z) + k1
                            index_j = i2*len(y)*len(z) + j2*len(z) + k2
                            
                            x[(i1,i2,j1,j2,k1,k2)] += A * 2**(-k1) * 2**(-k2)  
                            
                            
                             
                            # converting into a upper triangular matrix
                            if index_i > index_j:
                                Q[(index_j, index_i)] += A * 2**(-k1) * 2**(-k2)  
                            else:
                                Q[(index_i, index_j)] += A * 2**(-k1) * 2**(-k2)     
                            

print("this is the input dictionary for DWave:")
                
print(Q)
                   
print("this is the dictionary for the user:")
print("order of the basis: (time time', asset, asset', digit, digit' )")              
           
    
    
print(x)


this is the input dictionary for DWave:
defaultdict(<function <lambda> at 0x000001D8E3BA8B80>, {(0, 0): 0.48804778698972756, (1, 1): -0.0059761065051362194, (2, 2): -0.06548805325256811, (3, 3): -0.048369026626284055, (4, 4): 1.2240869866248285, (5, 5): 0.36204349331241426, (6, 6): 0.11852174665620713, (7, 7): 0.043635873328103564, (8, 8): -0.07169990537098274, (9, 9): -0.28584995268549135, (10, 10): -0.20542497634274567, (11, 11): -0.11833748817137284, (12, 12): 1.0520194034877233, (13, 13): 0.27600970174386164, (14, 14): 0.07550485087193082, (15, 15): 0.02212742543596541, (16, 16): -1.1198176539709883, (17, 17): -0.8099088269854942, (18, 18): -0.4674544134927471, (19, 19): -0.24935220674637354, (20, 20): 1.5340379295867, (21, 21): 0.51701896479335, (22, 22): 0.19600948239667498, (23, 23): 0.08237974119833749, (24, 24): 0.8174138607162544, (25, 25): 0.15870693035812722, (26, 26): 0.01685346517906361, (27, 27): -0.007198267410468195, (28, 28): 0.399403672397387, (29, 29): -0.0502981638

In [127]:
aa = defaultdict(float)

aa["jose"]+= 25

print(aa)

defaultdict(<class 'float'>, {'jose': 25.0})
